# Mac Tensorflow setup

In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.python.compiler.mlcompute import mlcompute

import time

print(tf.__version__)
# GPGPU now can not work with eager configuration
# tf.compat.v1.disable_eager_execution()
# results in only CPUs since not configured to see apple ones
print(tf.config.list_physical_devices())
# the way to force CPU. otherwise uses gpu
mlcompute.set_mlc_device(device_name='gpu')

# results in nothing since not configured to see apple ones
print([x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU'])

# auxiliary libs to communicate with apple GPUs
print("is_apple_mlc_enabled %s" % mlcompute.is_apple_mlc_enabled())
print("is_tf_compiled_with_apple_mlc %s" % mlcompute.is_tf_compiled_with_apple_mlc())

2.4.0-rc0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[]
is_apple_mlc_enabled True
is_tf_compiled_with_apple_mlc True


2021-08-20 23:21:13.223637: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-20 23:21:13.224635: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Core code

In [2]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os

## Settings

In [3]:
INPUT_SHAPE = (256, 256, 3)
MASK_SIZE = (32, 32, 3)

SEED = 13
SOURCE_FOLDER = "source_images"
RESHAPED_IMAGES_FOLDER = "reshaped_images"
TARGET_FOLDER = "processed_images"
CROPPED_FOLDER = "cropped_images"

## Support functions

In [4]:
def log(text):
    print(text)
    
def log_separator():
    log("========================================================")

## Image preprocessing

In [5]:
def get_mask_params():
    shape = INPUT_SHAPE[:2]
    x = y = int(shape[0]/2 - MASK_SIZE[0]/2)
    w = h = MASK_SIZE[0]
    return ((x,y),(w,h))

def get_mask():
    shape = INPUT_SHAPE[:2]
    ((x,y),(w,h)) = get_mask_params()
    
    mask = np.ones(INPUT_SHAPE[:2], np.uint8) * 255
    mask[y:y+h,x:x+w]=0
    
    return mask

def invert_mask(mask):
    mask = np.where(mask == 0, 254, mask)
    mask = np.where(mask == 255, 0, mask)
    mask = np.where(mask == 254, 255, mask)
    return mask

def crop_images(source_folder=SOURCE_FOLDER, target_folder=CROPPED_FOLDER):
    ((x,y),(w,h)) = get_mask_params()
    
    for filename in os.listdir(source_folder):
        img = cv2.imread(os.path.join(source_folder, filename))
        if img is not None:
            cropped_image = img[y:y+h,x:x+w]
            cv2.imwrite(os.path.join(target_folder, filename), cropped_image)
            
def run_to_see_images_with_masks(train_generator, number_of_images):
    output = []
    counter = 1
    for v in train_generator:
        if counter >= number_of_images:
            break
        output.append(v)
        counter += 1
    return output

In [6]:
mask = get_mask()
inverted_mask = invert_mask(mask)

# reshape source images
reshape_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1/255
)
reshape_generator = reshape_datagen.flow_from_directory(
    directory=SOURCE_FOLDER,
    save_to_dir=RESHAPED_IMAGES_FOLDER+"/images",
    batch_size=1,
    class_mode=None,
    target_size=INPUT_SHAPE[:-1],
    seed=SEED
)
run_to_see_images_with_masks(reshape_generator, reshape_generator.samples)

# crop images
source_full_path = RESHAPED_IMAGES_FOLDER+"/images"
crop_images(source_folder=source_full_path, target_folder=CROPPED_FOLDER+"/images")

# image preprocessing
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    preprocessing_function=lambda img: cv2.bitwise_and(img, img, mask=mask)
)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
)
image_generator = image_datagen.flow_from_directory(
    directory=SOURCE_FOLDER,
    save_to_dir=TARGET_FOLDER,
    batch_size=1,
    class_mode=None,
    save_prefix="direct_",
    target_size=INPUT_SHAPE[:-1],
    seed=SEED
)
mask_generator = mask_datagen.flow_from_directory(
    directory=CROPPED_FOLDER,
    save_to_dir=TARGET_FOLDER,
    batch_size=1,
    class_mode=None,
    save_prefix="masked_",
    target_size=MASK_SIZE[:-1],
    seed=SEED
)
train_generator = zip(image_generator, mask_generator)
number_of_images = image_generator.samples
print(f"Number of images found: {number_of_images}")

output = run_to_see_images_with_masks(train_generator, number_of_images)

Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Number of images found: 5


## Model construction

In [7]:
def extract_base_model(input_shape):
    log(f"Base model input shape: {str(input_shape)}")
    
    return tf.keras.applications.EfficientNetB4(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )

def get_channel_wise_layer(number_of_filters, kernel_size):
    log(f"Channel wise layer properties:\r\n   number of filters:\
        {str(number_of_filters)}\r\n.  kernel size:\
        {str(kernel_size)}")
    
    return tf.keras.layers.SeparableConv2D(
        filters=number_of_filters,
        kernel_size=kernel_size
    )

def get_deconvolution_layer(number_of_filters, kernel_size):
    log(f"Deconvolution layer properties:\r\n   number of filters:\
        {str(number_of_filters)}\r\n.  kernel size:\
        {str(kernel_size)}")
    
    return tf.keras.layers.Conv2DTranspose(
        filters=number_of_filters,
        kernel_size=kernel_size
    )

def create_model(list_of_layers):
    model = tf.keras.Sequential()
    
    for layer in list_of_layers:
        model.add(layer)
        
    return model

def extract_model(input_shape):
    log_separator()
    base_model = extract_base_model(input_shape)
    
    channel_wise_layer = get_channel_wise_layer(
        number_of_filters=base_model.layers[-1].output_shape[-1],
        kernel_size=(1,1)
    )
    
    deconvolution_1 = get_deconvolution_layer(
        number_of_filters=700,
        kernel_size=(3,3)
    )
    deconvolution_2 = get_deconvolution_layer(
        number_of_filters=224,
        kernel_size=(8,8)
    )
    deconvolution_3 = get_deconvolution_layer(
        number_of_filters=56,
        kernel_size=(9,9)
    )
    deconvolution_4 = get_deconvolution_layer(
        number_of_filters=3,
        kernel_size=(8,8)
    )
    
#     flatten_layer_1 = tf.keras.layers.Flatten()

    model = create_model([
        base_model,
        channel_wise_layer,
        deconvolution_1,
        deconvolution_2,
        deconvolution_3,
        deconvolution_4,
#         flatten_layer_1,
    ])
    
    model.summary()
    log_separator()
    return model

In [8]:
model = extract_model(INPUT_SHAPE)

Base model input shape: (256, 256, 3)
Channel wise layer properties:
   number of filters:        1792
.  kernel size:        (1, 1)
Deconvolution layer properties:
   number of filters:        700
.  kernel size:        (3, 3)
Deconvolution layer properties:
   number of filters:        224
.  kernel size:        (8, 8)
Deconvolution layer properties:
   number of filters:        56
.  kernel size:        (9, 9)
Deconvolution layer properties:
   number of filters:        3
.  kernel size:        (8, 8)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb4 (Functional)  (None, 8, 8, 1792)        17673823  
_________________________________________________________________
separable_conv2d (SeparableC (None, 8, 8, 1792)        3214848   
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 10, 10, 700)       11290300  
________

In [9]:
def my_loss_fn(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

model.compile(loss=my_loss_fn)

In [10]:
model.fit(
    train_generator,
    steps_per_epoch=2000,
    epochs=50,
    verbose=True
)

2021-08-20 23:21:37.056156: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
  20/2000 [..............................] - ETA: 46:15 - loss: 381114.2256

KeyboardInterrupt: 